In [1]:
using Parameters                     # @unpack
using QuantEcon:rouwenhorst,tauchen  # Rouwenhorst + Tauchen
using Roots:find_zero                # root finding
using SparseArrays                   # sparse matrices
using IterativeSolvers:powm!         # largest eigenvector + associated eigenvector
using Plots                          # Plots
using LinearAlgebra
using NBInclude                      # including notebooks instead of jl files
using JuMP                           # Optimization package
import Ipopt                         # related to JuMP
import Test                          # related to JuMP
using MAT                            # writing a .m file
using DataFrames
using Printf


In [2]:
@nbinclude("Structures_SW.ipynb"); 
@nbinclude("Utils_SW.ipynb");
@nbinclude("SolveAiyagari_SW.ipynb");
@nbinclude("Truncation_SW.ipynb");
@nbinclude("Ramsey_SW.ipynb");
@nbinclude("ToDynare_SW.ipynb");

## Main Specification 

In [3]:
economy = Economy(
    α      = 0.36,             # capital share
    β      = 0.9891196834817012, 
    δ      = 0.025,            # capital depreciation
    ϕ      = 0.5,#1.0,         # Frisch elasticity 0.5
    χ      = 0.061,             
    γ      = 1.0,              # inverse of IES #2.0 Curvature utility is 1.2 in monetary paper 
    τk     = 0.36,             # capital tax
    τl     = 0.28,             # labor tax
    Tt     = 0.114,            # lump-sum transfer
    na     = 100,              # nb of asset grid points
    a_min  = 1e-9,             # minimal asset holding
    a_max  = 1000.0,           # maximal asset holding
    curv_a = 4,                # curvature of the exponential asset grid 
    ny     = 5,                # nb of producivity states
    ρy     = 0.9885,           # quarterly persistence of productivity
    σy     = 0.154             # quarterly SE of productivity innovation
    );



# the function considers the same market power and change price adjustment cost to match the desired slope
economy = Economy(economy, 
    slope_p=0.00,
    slope_w=0.03);


solution = steady(economy,initL=0.3326818364888149,tolL=1e-10,maxiterL=100);

# Setting truncation length
N = 5            # uniform truncation
refiNs = fill(N+1, economy.ny)   # refined truncation

# Computing the truncated model
truncatedModel = TruncatedModel(N,refiNs,solution,economy);
print("Total number of positive-size histories: ",truncatedModel.Ntot);

ramsey = Ramsey(truncatedModel,solution,economy)
Write_Dynare("../dynamics/toDynare_SW.mat",ramsey,solution,economy)
(print_dict∘describe_solution)(solution,economy)


iteration: 50 0.04130553693089408
iteration: 100 0.08136691559381093
iteration: 150 0.009136032373954476
iteration: 200 0.0031241180556728894
iteration: 250 0.001210211520891905
iteration: 300 0.0004769009155005794
iteration: 350 0.00018666553750325745
iteration: 400 7.214021667376306e-5
iteration: 450 2.753478913081394e-5
iteration: 500 1.040351269704885e-5
iteration: 550 3.9017281700172115e-6
iteration: 600 1.4561010836259808e-6
iteration: 650 5.418122109605533e-7
iteration: 700 2.013061683525899e-7
iteration: 750 7.475290447810407e-8
iteration: 800 2.7758689328635857e-8
iteration: 850 1.031055105828631e-8
Solved in 900 iterations
(iterL, L0, abs(L0 - old_L)) = (1, 0.3326818354670531, 1.0217617862906536e-9)
Solved in 50 iterations
(iterL, L0, abs(L0 - old_L)) = (2, 0.33268183579344324, 3.2639013713975373e-10)
Solved in 50 iterations
(iterL, L0, abs(L0 - old_L)) = (3, 0.33268183612669566, 3.332524256549618e-10)
Solved in 50 iterations
(iterL, L0, abs(L0 - old_L)) = (4, 0.3326818363124

In [4]:
################################################
#### Wealth distribution in the model (Main Spec)
################################################

ww = [repeat(economy.aGrid,economy.ny) vec(solution.stationaryDist)] ;
ww2 = sortslices(ww, dims=1) ; 
vW = round.(wealthD(ww2,5), digits=2) ;

# Table 2 
df_quartile= DataFrame(Quartile=vec(round.(wealthD(ww2,5)[:,2]*100, digits=1)) ) ;
df_Gini= DataFrame(Gini = round.(Gini(solution.ga, solution.stationaryDist),digits=2)) ;
df_Top5= DataFrame(Top5 =round.(wealthD(ww2,20)[:,2]*100, digits=1)[end]) ;

display(df_quartile)
display(df_Top5)
display(df_Gini)


Row,Quartile
,Float64
1,0.0
2,0.3
3,5.4
4,18.7
5,75.6


Row,Top5
,Float64
1,34.5


Row,Gini
,Float64
1,0.73
